In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [3]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [4]:
# Load data
market_df = pd.read_excel("CL_Datasim.xlsx", engine="openpyxl")
tweets_df = pd.read_excel("merged_tweets.xlsx", engine="openpyxl")

In [5]:
# Parse datetime
market_df["Dates"] = pd.to_datetime(market_df["Dates"])
tweets_df["Timestamp"] = pd.to_datetime(tweets_df["Timestamp"], utc=True).dt.tz_convert(None)

In [6]:
# Compute Mid_Price
market_df["Mid_Price"] = market_df[["Mid_Open", "Mid_High", "Mid_Low", "Mid_Close"]].mean(axis=1)

In [7]:
# Set Dates as index for fast access
market_df.set_index("Dates", inplace=True)

In [8]:
# Time horizons to evaluate impact
time_horizons = {
    "MI_1min": timedelta(minutes=1),
    "MI_5min": timedelta(minutes=5),
    "MI_15min": timedelta(minutes=15),
    "MI_30min": timedelta(minutes=30),
    "MI_1h": timedelta(hours=1),
    "MI_2h": timedelta(hours=2),
    "MI_4h": timedelta(hours=4),
    "MI_8h": timedelta(hours=8),
    "MI_12h": timedelta(hours=12),
    "MI_1d": timedelta(days=1),
    "MI_2d": timedelta(days=2),
    "MI_3d": timedelta(days=3),
    "MI_4d": timedelta(days=4),
}

In [ ]:
# Calculate market impact for each tweet
results = []

for _, row in tweets_df.iterrows():
    tweet_time = row["Timestamp"]
    tweet_id = row["tweet_id"]

    # Find closest earlier timestamp
    if tweet_time in market_df.index:
        base_time = tweet_time
    else:
        base_time = market_df.index[market_df.index <= tweet_time].max()
        if pd.isna(base_time):
            continue

    try:
        base_price = market_df.loc[base_time, "Mid_Price"]
    except KeyError:
        continue

    impact_row = {
        "tweet_ID": tweet_id,
        "Timestamp": tweet_time,
        "Tweet": row["Tweet"],
        "Mid_Price": base_price
    }

    for label, delta in time_horizons.items():
        future_time = base_time + delta
        try:
            future_price = market_df.loc[future_time, "Mid_Price"]
            impact_row[label] = ((future_price - base_price) / base_price) * 10000  # in bps
        except KeyError:
            impact_row[label] = np.nan

    results.append(impact_row)

In [11]:

# Save to Excel
impact_df = pd.DataFrame(results)
impact_df.to_excel("tweet_market_impact.xlsx", index=False)